In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Lecture du fichier CSV contenant les ID des acteurs ou des réalisateurs
df_pers = pd.read_csv('ID_Acteurs.csv', sep=',')

# Définir le type de personnalité à rechercher ("star" pour acteur ou "director" pour réalisateur)
pers_type = "star"

# Enregistrement du temps de début pour mesurer la durée d'exécution du script
start_time = time.time()

# Initialisation des variables necessaires au suivi de l'avancement
pourcent = 4
total_pers = len(df_pers)
id = 0

# Création d'une session requests pour maintenir la persistance entre les requêtes
session = requests.Session()

# Configuration des cookies nécessaires pour la session (A remplir avec vos propre cookies)
cookies = {
    'session-token': '',
    'at-main': '',
    'sess-at-main': '',
    'x-main': '',
    'ci': '',
    'uu': '',
    'session-id': '',
    'lc-main': '',
    'session-id-time': '',
    'ubid-main': '',
}
session.cookies.update(cookies)

# URL et identifiants necessaires à la connexion au site IMDB Pro
login_url = 'https://secure.imdb.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.imdb.com%2Fregistration%2Fap-signin-handler%2Fimdb_pro_us&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=imdb_pro_us&openid.mode=checkid_setup&siteState=eyJvcGVuaWQuYXNzb2NfaGFuZGxlIjoiaW1kYl9wcm9fdXMiLCJyZWRpcmVjdFRvIjoiaHR0cHM6Ly9wcm8uaW1kYi5jb20vYWNjb3VudC9jYW5jZWw_cmVmXz10dF9wdWJfZnRyX2NhbmMifQ&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0'
login_payload = {
    'username': 'identifiant imdb pro',
    'password': 'mot de passe imdbpro'
}
response = session.post(login_url, data=login_payload)

# Configuration des en-têtes de la requête pour imiter un navigateur web
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'}

# Définition de la fonction pour le scraping d'IMDB Pro
def imdb_scrap(i):
    global pourcent, id, total_pers
    id += 1
    STARmeter = None

    # Récupération du nom de la personnalité.
    person = df_pers.loc[df_pers[f'{pers_type}_id'] == i, pers_type].values[0]

    try:
        # Construction et requête de l'URL de la page IMDB Pro de la personnalité.
        url = f'https://pro.imdb.com/name/{i}/overview'
        res = session.get(url, allow_redirects=True, headers=HEADERS)

        # affichage de l'avancement en pourcentage.
        pourcent += 1
        if pourcent == 5:
            print('Execution :', round((id/total_pers)*100, 2), '%')
            pourcent = 0

        # Vérification du statut de la réponse
        if res.status_code != 200:
            return i, "0", "0"
        if res.url != url:
            return i, "0", "0"
        
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(res.content, 'lxml')

        # Recherche du classement STARmeter sur la page
        STARmeter = soup.find('span', class_= 'a-size-base a-color-error a-text-bold')
        if STARmeter:
            STARmeter = STARmeter.text.strip().replace(',', '')
        else:
            STARmeter = soup.find('span', class_= 'a-size-base a-color-success a-text-bold')
            if STARmeter:
                STARmeter = STARmeter.text.strip().replace(',', '')

    # Gestion des exceptions liées aux requêtes HTTP    
    except requests.exceptions.RequestException:
        return i, "0", "0"
    
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    return (
        i,
        person if person else "0",
        STARmeter if STARmeter else "0",
    )

# Création d'un DataFrame vide pour stocker les résultats
df = pd.DataFrame(columns=['ID', pers_type, 'STARmeter'])

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=6) as executor:
    # Soumission des tâches de scraping pour chaque ID de film entre 1 et la variable total_film
    tests = [executor.submit(imdb_scrap, i) for i in df_pers[f'{pers_type}_id']]
    for test in tests:
        # Récupération des résultats et ajout au DataFrame si le titre est différent de "0"
        i, person, STARmeter = test.result()
        if person != "0":
            df.loc[i] = [i, person, STARmeter]

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv(f'IMDB_PRO_{pers_type}.csv', index=False)

# Apperçu du DataFrame final
display(df)





